In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import model
from datetime import datetime
from datetime import timedelta
import requests
import urllib
import lxml
from bs4 import BeautifulSoup
from bs4 import NavigableString
import autoencoder
# from plotly import py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [2]:
url="https://finance.yahoo.com/quote/fb/history?p=fb"
request = requests.get(url)
soup = BeautifulSoup(request.text, 'html.parser')
table_big = soup.find('table', attrs={'data-test':'historical-prices'})
soup.find('tfoot').decompose()

In [3]:
print(table_big)

<table class="W(100%) M(0)" data-reactid="31" data-test="historical-prices"><thead data-reactid="32"><tr class="C($c-fuji-grey-j) Fz(xs) Ta(end)" data-reactid="33"><th class="Ta(start) W(100px) Fw(400) Py(6px)" data-reactid="34"><span data-reactid="35">Date</span></th><th class="Fw(400) Py(6px)" data-reactid="36"><span data-reactid="37">Open</span></th><th class="Fw(400) Py(6px)" data-reactid="38"><span data-reactid="39">High</span></th><th class="Fw(400) Py(6px)" data-reactid="40"><span data-reactid="41">Low</span></th><th class="Fw(400) Py(6px)" data-reactid="42"><span data-reactid="43">Close*</span></th><th class="Fw(400) Py(6px)" data-reactid="44"><span data-reactid="45">Adj Close**</span></th><th class="Fw(400) Py(6px)" data-reactid="46"><span data-reactid="47">Volume</span></th></tr></thead><tbody data-reactid="48"><tr class="BdT Bdc($c-fuji-grey-c) Ta(end) Fz(s) Whs(nw)" data-reactid="49"><td class="Py(10px) Ta(start) Pend(10px)" data-reactid="50"><span data-reactid="51">Nov 05,

In [4]:
missing_values = ["n/a", "na", "-","NaN"]
rs = pd.read_html(str(table_big), na_values = missing_values)
rs = rs[0].to_json(orient='records')
print(rs)

[{"Date":"Nov 05, 2018","Open":150.1,"High":150.19,"Low":147.44,"Close*":148.68,"Adj Close**":148.68,"Volume":15897400},{"Date":"Nov 02, 2018","Open":151.8,"High":154.13,"Low":148.96,"Close*":150.35,"Adj Close**":150.35,"Volume":24708700},{"Date":"Nov 01, 2018","Open":151.52,"High":152.75,"Low":149.35,"Close*":151.75,"Adj Close**":151.75,"Volume":25640800},{"Date":"Oct 31, 2018","Open":155.0,"High":156.4,"Low":148.96,"Close*":151.79,"Adj Close**":151.79,"Volume":60101300},{"Date":"Oct 30, 2018","Open":139.94,"High":146.64,"Low":139.74,"Close*":146.22,"Adj Close**":146.22,"Volume":50528300},{"Date":"Oct 29, 2018","Open":148.5,"High":148.83,"Low":139.03,"Close*":142.09,"Adj Close**":142.09,"Volume":31336800},{"Date":"Oct 26, 2018","Open":145.82,"High":149.0,"Low":143.8,"Close*":145.37,"Adj Close**":145.37,"Volume":31303300},{"Date":"Oct 25, 2018","Open":147.73,"High":152.21,"Low":147.0,"Close*":150.95,"Adj Close**":150.95,"Volume":22105700},{"Date":"Oct 24, 2018","Open":154.28,"High":154

In [5]:
df = pd.read_json(rs)

In [6]:
df = df[['Date','Open','High','Low','Close*','Adj Close**']]
df = df.sort_values(by='Date', ascending=1)
df = df.dropna()
date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()
# df = df.apply(pd.to_numeric, args=('coerce',))
dfori = df.apply(pd.to_numeric, args=('coerce',))
print (df)

         Date    Open    High     Low  Close*  Adj Close**
99 2018-06-15  195.79  197.07  194.64  195.85       195.85
98 2018-06-18  194.80  199.58  194.13  198.31       198.31
97 2018-06-19  196.24  197.96  193.79  197.49       197.49
96 2018-06-20  199.10  203.55  198.81  202.00       202.00
95 2018-06-21  202.76  203.39  200.09  201.50       201.50
94 2018-06-22  201.16  202.24  199.31  201.74       201.74
93 2018-06-25  200.00  200.00  193.11  196.35       196.35
92 2018-06-26  197.60  199.10  196.23  199.00       199.00
91 2018-06-27  199.18  200.75  195.80  195.84       195.84
90 2018-06-28  195.18  197.34  193.26  196.23       196.23
89 2018-06-29  197.32  197.60  193.96  194.32       194.32
88 2018-07-02  193.37  197.45  192.22  197.36       197.36
87 2018-07-03  194.55  195.40  192.52  192.73       192.73
86 2018-07-05  194.74  198.65  194.03  198.45       198.45
85 2018-07-06  198.45  203.64  197.70  203.23       203.23
84 2018-07-09  204.93  205.80  202.12  204.74       204.

In [7]:
data1 = go.Scatter(x=df.Date, y=dfori['Open'], name='Open')
data2 = go.Scatter(x=df.Date, y=dfori['High'], name='High')
data3 = go.Scatter(x=df.Date, y=dfori['Low'], name='Low')
data4 = go.Scatter(x=df.Date, y=dfori['Close*'], name='Close')
data5 = go.Scatter(x=df.Date, y=dfori['Adj Close**'], name='Adj Close**')

data = [data1, data2, data3, data4, data5]

iplot(data)

In [8]:
minmax = MinMaxScaler().fit(df.iloc[:, 1:].astype('float32'))
minmax.transform(df.iloc[:, 1:].astype('float32'))

C:\Users\admin\Anaconda3\envs\finalYearProject\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning:

Data with input dtype float32 were all converted to float64 by MinMaxScaler.



array([[0.73700174, 0.70061143, 0.7391015 , 0.71290289, 0.71290289],
       [0.72393773, 0.73548216, 0.73232326, 0.74552445, 0.74552445],
       [0.74294014, 0.71297597, 0.72780423, 0.73465066, 0.73465066],
       [0.78068098, 0.79063639, 0.79452412, 0.79445698, 0.79445698],
       [0.82897853, 0.78841349, 0.81153633, 0.78782656, 0.78782656],
       [0.8078649 , 0.77243692, 0.80116952, 0.79100923, 0.79100923],
       [0.79255738, 0.74131708, 0.71876659, 0.71953331, 0.71953331],
       [0.76088684, 0.7288137 , 0.76023382, 0.75467446, 0.75467446],
       [0.78173649, 0.75173665, 0.75451887, 0.71277016, 0.71277016],
       [0.72895211, 0.70436232, 0.72076012, 0.71794187, 0.71794187],
       [0.75719195, 0.70797457, 0.73006385, 0.69261382, 0.69261382],
       [0.70506721, 0.70589053, 0.70693778, 0.73292669, 0.73292669],
       [0.72063871, 0.67741036, 0.71092506, 0.67152893, 0.67152893],
       [0.723146  , 0.72256179, 0.7309941 , 0.74738096, 0.74738096],
       [0.7721034 , 0.79188668, 0.

In [9]:
minmax = MinMaxScaler().fit(df.iloc[:, 1:].astype('float32'))
df_log = minmax.transform(df.iloc[:, 1:].astype('float32'))
df_log = pd.DataFrame(df_log)
df_log.head()

C:\Users\admin\Anaconda3\envs\finalYearProject\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning:

Data with input dtype float32 were all converted to float64 by MinMaxScaler.



,0,1,2,3,4
0,0.737002,0.700611,0.739101,0.712903,0.712903
1,0.723938,0.735482,0.732323,0.745524,0.745524
2,0.742940,0.712976,0.727804,0.734651,0.734651
3,0.780681,0.790636,0.794524,0.794457,0.794457
4,0.828979,0.788413,0.811536,0.787827,0.787827


In [10]:
thought_vector = autoencoder.reducedimension(df_log.values, 16, 0.001, 128, 100)

epoch: 10 loss: 0.29233962 time: 0.002998828887939453
epoch: 20 loss: 0.2923381 time: 0.003012418746948242
epoch: 30 loss: 0.29233542 time: 0.003999948501586914
epoch: 40 loss: 0.29233086 time: 0.0020008087158203125
epoch: 50 loss: 0.29232305 time: 0.0009996891021728516
epoch: 60 loss: 0.2923096 time: 0.0010006427764892578
epoch: 70 loss: 0.2922859 time: 0.002000093460083008
epoch: 80 loss: 0.29224312 time: 0.0019989013671875
epoch: 90 loss: 0.2921622 time: 0.002001047134399414
epoch: 100 loss: 0.2919943 time: 0.002000093460083008


In [11]:
thought_vector.shape

(100, 16)

In [12]:
num_layers = 1
size_layer = 128
timestamp = 7
epoch = 500
dropout_rate = 1
future_day = 14

In [13]:
tf.reset_default_graph()
modelnn = model.Model(0.01, num_layers, df_log.shape[1], size_layer, df_log.shape[1], dropout_rate)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for i in range(epoch):
    init_value = np.zeros((1, num_layers * 2 * size_layer))
    total_loss = 0
    for k in range(0, (df_log.shape[0] // timestamp) * timestamp, timestamp):
        batch_x = np.expand_dims(df_log.iloc[k: k + timestamp, :].values, axis = 0)
        batch_y = df_log.iloc[k + 1: k + timestamp + 1, :].values
        last_state, _, loss = sess.run([modelnn.last_state, 
                                        modelnn.optimizer, 
                                        modelnn.cost], feed_dict={modelnn.X: batch_x, 
                                                                  modelnn.Y: batch_y, 
                                                                  modelnn.hidden_layer: init_value})
        loss = np.mean(loss)
        init_value = last_state
        total_loss += loss
    total_loss /= (df_log.shape[0] // timestamp)
    if (i + 1) % 100 == 0:
        print('epoch:', i + 1, 'avg loss:', total_loss)

C:\Users\admin\Anaconda3\envs\finalYearProject\lib\site-packages\tensorflow\python\client\session.py:1662: UserWarning:

An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).



epoch: 100 avg loss: 0.008820399924713587
epoch: 200 avg loss: 0.00635327013774908
epoch: 300 avg loss: 0.00806933082640171
epoch: 400 avg loss: 0.008960277457455439
epoch: 500 avg loss: 0.0056926387062828454


In [14]:
output_predict = np.zeros((df_log.shape[0] + future_day, df_log.shape[1]))
output_predict[0, :] = df_log.iloc[0, :]
upper_b = (df_log.shape[0] // timestamp) * timestamp
init_value = np.zeros((1, num_layers * 2 * size_layer))
for k in range(0, (df_log.shape[0] // timestamp) * timestamp, timestamp):
    out_logits, last_state = sess.run([modelnn.logits, modelnn.last_state], feed_dict = {modelnn.X:np.expand_dims(df_log.iloc[k: k + timestamp, :], axis = 0),
                                     modelnn.hidden_layer: init_value})
    init_value = last_state
    output_predict[k + 1: k + timestamp + 1, :] = out_logits
    
out_logits, last_state = sess.run([modelnn.logits, modelnn.last_state], feed_dict = {modelnn.X:np.expand_dims(df_log.iloc[upper_b: , :], axis = 0),
                                     modelnn.hidden_layer: init_value})
init_value = last_state
output_predict[upper_b + 1: df_log.shape[0] + 1, :] = out_logits
df_log.loc[df_log.shape[0]] = out_logits[-1, :]
date_ori.append(date_ori[-1]+timedelta(days=1))

In [15]:
for i in range(future_day - 1):
    out_logits, last_state = sess.run([modelnn.logits, modelnn.last_state], feed_dict = {modelnn.X:np.expand_dims(df_log.iloc[-timestamp:, :], axis = 0),
                                     modelnn.hidden_layer: init_value})
    init_value = last_state
    output_predict[df_log.shape[0], :] = out_logits[-1, :]
    df_log.loc[df_log.shape[0]] = out_logits[-1, :]
    date_ori.append(date_ori[-1]+timedelta(days=1))

In [16]:
df_log = minmax.inverse_transform(output_predict)
date_ori=pd.Series(date_ori).dt.strftime(date_format='%Y-%m-%d').tolist()

In [17]:
x_range_original = np.arange(dfori.shape[0])
x_range_future = np.arange(df_log.shape[0])

data4 = go.Scatter(x=x_range_original, y=dfori['Close*'], name='True Close')
data5 = go.Scatter(x=x_range_future, y=df_log[:, 3], name='Predicted Close')

data = [data4, data5]

iplot(data)

In [23]:
data21 = go.Scatter(x=x_range_original, y=dfori['Open'], name='True Open')
data22 = go.Scatter(x=x_range_future, y=df_log[:, 0], name='Predicted Open')
data23 = go.Scatter(x=x_range_original, y=dfori['High'], name='True High')
data24 = go.Scatter(x=x_range_future, y=df_log[:, 1], name='Predicted High')
data25 = go.Scatter(x=x_range_original, y=dfori['Low'], name='True Low')
data26 = go.Scatter(x=x_range_future, y=df_log[:, 2], name='Predicted Low')
data27 = go.Scatter(x=x_range_original, y=dfori['Close*'], name='True Close')
data28 = go.Scatter(x=x_range_future, y=df_log[:, 3], name='Predicted Close')
data29 = go.Scatter(x=x_range_original, y=dfori['Adj Close**'], name='True Adj Close')
data30 = go.Scatter(x=x_range_future, y=df_log[:, 4], name='Predicted Adj Close')

data2 = [data21, data22, data23, data24, data25, data26, data27, data28, data29, data30]

iplot(data2)

In [24]:
sess.close()